In [4]:
# Exp_Group1_SORT_DeepSORT.ipynb

# ============================================
# 📌 設定 & 核心模組載入
# ============================================
import sys
import traceback
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
import torch
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# --- 1. 定義根目錄 ---
root_path = Path().resolve()
while root_path.name != "YOLO11_Experiment" and root_path != root_path.parent:
    root_path = root_path.parent

sys.path.append(str(root_path / "utils"))

# --- 2. 匯入工具 ---
from source_utils import detect_source_type
from yolo_detector import load_model, run_predict_on_validation_set
from evaluation_metrics import compute_class_stability_with_id, compute_mot_metrics, compute_class_stability
from authentic_tracking_launcher import run_authentic_tracker # Group 1 專用
from voting_mechanism import run_class_voting

# ============================================
# 🛠️ 實驗設定 (Group 1)
# ============================================

# 全域時間戳
BATCH_TIMESTAMP = datetime.now().strftime("%Y%m%d-%H%M%S")

DETECTOR_WEIGHT = "weights/combined_freeze10.pt"

CONF_THRESHOLDS = [0.3, 0.4, 0.5]
IOU_THRESHOLDS = [0.3, 0.7]

TARGET_DATASETS = [
    "20230417_06_00230", "DSCN5480", "IMG_0004", "RCNX0029", "RCNX0062", 
    "RCNX0279", "group3_20240429_0248", "group5_20240429_0289", 
    "EK000061-20170904-1740", "IMG_0057"
]

TRACKERS = ['SORT', 'DeepSORT']

VOTING_SETTINGS = [
    {'method': 'majority', 'window': 3}, 
    {'method': 'majority', 'window': 5},
    {'method': 'majority', 'window': 10}, 
    {'method': 'majority', 'window': 'all'}, 
    {'method': 'weighted', 'window': 3}, 
    {'method': 'weighted', 'window': 5},
    {'method': 'weighted', 'window': 10}, 
    {'method': 'weighted', 'window': 'all'},
]

# 輸出路徑與 Log 檔名皆加上時間戳
EXP_GROUP_DIR = root_path / "results" / f"group1_sort_deepsort_{BATCH_TIMESTAMP}"
LOG_FILE_PATH = EXP_GROUP_DIR / f"batch_experiments_log_{BATCH_TIMESTAMP}.csv"

# ============================================
# ⚙️ 輔助函式
# ============================================
def log_experiment_result(log_path, data_dict):
    df_new = pd.DataFrame([data_dict])
    log_path.parent.mkdir(parents=True, exist_ok=True)
    if not log_path.exists():
        df_new.to_csv(log_path, index=False)
    else:
        df_new.to_csv(log_path, mode='a', header=False, index=False)

def calc_fps(latency_ms):
    return 1000.0 / latency_ms if latency_ms > 0 else 0.0

# ============================================
# 🧪 核心實驗邏輯
# ============================================
def run_dataset_experiment(dataset_name, model):
    
    current_time = datetime.now().strftime("%H%M%S") # 用於子資料夾區分
    
    print(f"\n{'='*60}")
    print(f"📦 正在處理資料集: {dataset_name} (Time: {current_time})")
    print(f"{'='*60}")

    raw_path = root_path / "data" / dataset_name
    if not raw_path.exists():
        print(f"❌ 找不到資料夾: {raw_path}")
        return

    source_path = detect_source_type(raw_path)
    gt_path = raw_path / 'gt' / 'gt.txt'
    
    # 使用全域定義的輸出目錄
    TEST_ROOT_DIR = EXP_GROUP_DIR

    for conf in CONF_THRESHOLDS:
        for iou in IOU_THRESHOLDS:
            print(f"\n🔹 [Config] Dataset: {dataset_name} | Conf: {conf} | IoU: {iou}")
            
            # -------------------------------------------------
            # 1. Detection Only
            # -------------------------------------------------
            print(f"   👉 [Condition 1] Detection Only...")
            det_output_dir = TEST_ROOT_DIR / f"{dataset_name}_Det_c{conf}_iou{iou}_{current_time}"
            
            det_csv_path, lat_det = run_predict_on_validation_set(
                model, source_path, det_output_dir, imgsz=640, conf=conf, iou=iou, half=True
            )
            
            fps_det = calc_fps(lat_det)
            print(f"      ⏱️ Detection Latency: {lat_det:.2f} ms | FPS: {fps_det:.1f}")
            
            stab_det = compute_class_stability(det_csv_path)
            
            log_experiment_result(LOG_FILE_PATH, {
                "dataset": dataset_name, "conf": conf, "iou": iou,
                "tracker": "None", 
                "method": "detection", 
                "window": "None",
                "MOTA": np.nan, 
                "IDF1": np.nan, 
                "IDSW": np.nan,
                "instability": stab_det.get('class_instability_rate', np.nan),
                "latency_ms": lat_det, "FPS": fps_det,
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            })
        
            # -------------------------------------------------
            # 2. Loop Trackers (SORT, DeepSORT)
            # -------------------------------------------------
            for tracker_type in TRACKERS:
                print(f"\n   👉 [Condition 2] Running Tracker: {tracker_type} (Tracking Only)...")
                
                track_output_dir = TEST_ROOT_DIR / f"{dataset_name}_{tracker_type}_c{conf}_iou{iou}_{current_time}"
                track_output_dir.mkdir(parents=True, exist_ok=True)
                
                # run_authentic_tracker 回傳 (df, 總秒數)
                df_track, total_seconds = run_authentic_tracker(
                    model, tracker_type, source_path, track_output_dir, imgsz=640, conf=conf, iou=iou, half=True
                )
                
                if df_track.empty:
                    print(f"      ⚠️ {tracker_type} 結果為空，跳過。")
                    continue
                    
                n_frames = df_track['frame_id'].nunique()
                lat_track = (total_seconds * 1000) / n_frames if n_frames > 0 else 0
                fps_track = calc_fps(lat_track)
                
                print(f"      ⏱️ Tracking Latency: {lat_track:.2f} ms | FPS: {fps_track:.1f}")
        
                csv_name_raw = f"{tracker_type}_raw.csv"
                csv_path_raw = track_output_dir / csv_name_raw
                df_track.to_csv(csv_path_raw, index=False)
                
                stab_raw = compute_class_stability_with_id(csv_path_raw)
                mot_raw = compute_mot_metrics(csv_path_raw, gt_path) or {}
                
                log_experiment_result(LOG_FILE_PATH, {
                    "dataset": dataset_name, "conf": conf, "iou": iou,
                    "tracker": tracker_type, 
                    "method": "tracking", 
                    "window": "None",
                    "MOTA": mot_raw.get('MOTA', np.nan), 
                    "IDF1": mot_raw.get('IDF1', np.nan), 
                    "IDSW": mot_raw.get('IDSW', np.nan),
                    "instability": stab_raw.get('class_instability_rate_id', np.nan),
                    "latency_ms": lat_track, "FPS": fps_track,
                    "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                })
                
                # 執行投票
                print(f"   👉 [Condition 3] Running Voting Analysis...")
                for vote_cfg in VOTING_SETTINGS:
                    method, window = vote_cfg['method'], vote_cfg['window']
                    
                    df_vote, lat_vote = run_class_voting(df_track, method, window, is_realtime=True)
                    
                    total_lat = lat_track + lat_vote
                    total_fps = calc_fps(total_lat)
        
                    csv_name_vote = f"{tracker_type}_{method}_w{window}.csv"
                    csv_path_vote = track_output_dir / csv_name_vote
                    df_vote.to_csv(csv_path_vote, index=False)
        
                    stab_vote = compute_class_stability_with_id(csv_path_vote)
                    mot_vote = compute_mot_metrics(csv_path_vote, gt_path) or {}
                    
                    log_experiment_result(LOG_FILE_PATH, {
                        "dataset": dataset_name, "conf": conf, "iou": iou,
                        "tracker": tracker_type, 
                        "method": method, 
                        "window": window,
                        "MOTA": mot_vote.get('MOTA', np.nan), 
                        "IDF1": mot_vote.get('IDF1', np.nan), 
                        "IDSW": mot_vote.get('IDSW', np.nan),
                        "instability": stab_vote.get('class_instability_rate_id', np.nan),
                        "latency_ms": total_lat, "FPS": total_fps,
                        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    })

# ============================================
# 🚀 主程式入口
# ============================================
if __name__ == "__main__":
    print(f"📦 開始執行 Group 1 實驗 (SORT, DeepSORT)")
    print(f"📂 輸出目錄: {EXP_GROUP_DIR}")
    print(f"📝 Log 檔案: {LOG_FILE_PATH}")
    
    model = load_model(root_path / DETECTOR_WEIGHT)
    
    print("🔥 正在進行 GPU 暖身 (Warm-up)...")
    try:
        dummy_input = torch.zeros(1, 3, 640, 640).to(model.device)
        for _ in range(3):
            model(dummy_input, verbose=False)
        print("✅ 暖身完成！GPU 已準備就緒。")
    except Exception as e:
        print(f"⚠️ 暖身失敗 (可能是 CPU 模式): {e}")

    for dataset in TARGET_DATASETS:
        try:
            run_dataset_experiment(dataset, model)
        except Exception as e:
            print(f"❌ 資料集 {dataset} 發生錯誤: {e}")
            traceback.print_exc()
    print(f"\n🎉 Group 1 實驗完成！結果已存至: {LOG_FILE_PATH}")

📦 開始執行 Group 1 實驗 (SORT, DeepSORT)
📂 輸出目錄: D:\experiment\YOLO11_Experiment\results\group1_sort_deepsort_20260102-152236
📝 Log 檔案: D:\experiment\YOLO11_Experiment\results\group1_sort_deepsort_20260102-152236\batch_experiments_log_20260102-152236.csv
🔥 正在進行 GPU 暖身 (Warm-up)...
✅ 暖身完成！GPU 已準備就緒。

📦 正在處理資料集: 20230417_06_00230 (Time: 152237)

🔹 [Config] Dataset: 20230417_06_00230 | Conf: 0.3 | IoU: 0.3
   👉 [Condition 1] Detection Only...

✅ 儲存驗證集結果: D:\experiment\YOLO11_Experiment\results\group1_sort_deepsort_20260102-152236\20230417_06_00230_Det_c0.3_iou0.3_152237\20230417_06_00230_predictions.csv
   ⏱️ 偵測耗時: 38498.94 ms
      ⏱️ Detection Latency: 192.49 ms | FPS: 5.2

   👉 [Condition 2] Running Tracker: SORT (Tracking Only)...
🔄 初始化 SORT...
🖼️ 讀取圖片序列: 共 200 張
🚀 開始追蹤 (SORT) - 請耐心等候...
   Frame 190 | Dets: 1 | FPS: 5.06
   ✅ 完成。共 200 幀，總耗時 39.31 秒。
      ⏱️ Tracking Latency: 217.19 ms | FPS: 4.6
   👉 [Condition 3] Running Voting Analysis...
   ... 對 1 個目標進行 [majority] 投票 (Win: 3, Mode: Re

C:\Users\USER\anaconda3\envs\sort_deepsort\lib\site-packages\deep_sort_realtime\embedder\embedder_pytorch.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


   Frame 190 | Dets: 1 | FPS: 4.67
   ✅ 完成。共 200 幀，總耗時 42.63 秒。
      ⏱️ Tracking Latency: 230.43 ms | FPS: 4.3
   👉 [Condition 3] Running Voting Analysis...
   ... 對 1 個目標進行 [majority] 投票 (Win: 3, Mode: Real-time)
   ⏱️ 投票運算耗時: 19.98 ms (平均 0.1080 ms/frame)
   ... 對 1 個目標進行 [majority] 投票 (Win: 5, Mode: Real-time)
   ⏱️ 投票運算耗時: 19.61 ms (平均 0.1060 ms/frame)
   ... 對 1 個目標進行 [majority] 投票 (Win: 10, Mode: Real-time)
   ⏱️ 投票運算耗時: 20.06 ms (平均 0.1084 ms/frame)
   ... 對 1 個目標進行 [majority] 投票 (Win: all, Mode: Real-time)
   ⏱️ 投票運算耗時: 19.73 ms (平均 0.1066 ms/frame)
   ... 對 1 個目標進行 [weighted] 投票 (Win: 3, Mode: Real-time)
   ⏱️ 投票運算耗時: 81.20 ms (平均 0.4389 ms/frame)
   ... 對 1 個目標進行 [weighted] 投票 (Win: 5, Mode: Real-time)
   ⏱️ 投票運算耗時: 82.75 ms (平均 0.4473 ms/frame)
   ... 對 1 個目標進行 [weighted] 投票 (Win: 10, Mode: Real-time)
   ⏱️ 投票運算耗時: 79.49 ms (平均 0.4297 ms/frame)
   ... 對 1 個目標進行 [weighted] 投票 (Win: all, Mode: Real-time)
   ⏱️ 投票運算耗時: 84.08 ms (平均 0.4545 ms/frame)

🔹 [Config] Dataset: 2023041